<h1>Table des Matières<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Connection-avec-la-base-de-données" data-toc-modified-id="Connection-avec-la-base-de-données-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Connection avec la base de données</a></span></li><li><span><a href="#Question-19" data-toc-modified-id="Question-19-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Question 19</a></span></li><li><span><a href="#Question-20" data-toc-modified-id="Question-20-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Question 20</a></span></li></ul></div>

### Connection avec la base de données

In [1]:
import sqlite3 as sql
import pandas as pd 

In [2]:
# Création de la connection à la Database

database = "/Users/Gladou/DB_FAO.db"
connection = sql.connect(database)

In [3]:
# Exemple : 

query = ''' SELECT * FROM population '''
df = pd.read_sql_query(query, connection)
df.head()

,pays,code_pays,annee,population
0,Afghanistan,2,2013,30552000
1,Afrique du Sud,202,2013,52776000
2,Albanie,3,2013,3173000
3,Algérie,4,2013,39208000
4,Allemagne,79,2013,82727000


### Question 19 

**Écrivez les requêtes SQL permettant de connaître :**

**Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant, puis en termes de kcal par habitant.**

In [4]:
# La disponibilité en protéines est en g/personne/jour, pour l'avoir en kg, je divise cette dernière par 
# 1000. De ce fait, j'ai une dispo en kg/personne/jour. Ensuite, je la multiplie donc par 365 jours pour 
# l'avoir en kg/pers. 

# Comme il s'agit d'avoir ici un ratio par habitant, je divise donc cette dispo en kg/pers par le nombre 
# de personne. Ce qui revient finalement à multiplier par le nombre de pers et ainsi 'annuler' ce terme.
# De ce fait, lorsqu'il s'agit d'un ratio, on peut directement considérer la donnée en kg, en kcal, ou autre
# (suivant la question).   

query = '''SELECT pays, SUM((dispo_prot / 1000) * 365) AS dispo_prot_tot FROM dispo_alim 
            GROUP BY pays
            ORDER BY dispo_prot_tot DESC
            LIMIT 10 '''

df = pd.read_sql_query(query, connection)
df.head(10)

,pays,dispo_prot_tot
0,Islande,48.56690
1,Israël,46.72000
2,Lituanie,45.39140
3,Maldives,44.64680
4,Finlande,42.90940
5,Luxembourg,41.47860
6,Monténégro,40.84350
7,Pays-Bas,40.68290
8,Albanie,40.65005
9,Portugal,40.46755


In [5]:
query = ''' SELECT pays, SUM(dispo_alim_kcal_p_j * 365) AS dispo_kcal_tot FROM dispo_alim 
            GROUP BY pays
            ORDER BY dispo_kcal_tot DESC
            LIMIT 10 '''

df = pd.read_sql_query(query, connection)
df.head(10)

,pays,dispo_kcal_tot
0,Autriche,1376050.0
1,Belgique,1364005.0
2,Turquie,1353420.0
3,États-Unis d'Amérique,1343930.0
4,Israël,1317650.0
5,Irlande,1314730.0
6,Italie,1305970.0
7,Luxembourg,1292100.0
8,Égypte,1284070.0
9,Allemagne,1278595.0


**Pour l'année 2013, les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant.**

In [6]:
query = ''' SELECT pays, SUM((dispo_prot / 1000) * 365) AS dispo_prot_tot FROM dispo_alim 
            GROUP BY pays
            ORDER BY dispo_prot_tot ASC
            LIMIT 10 '''

df = pd.read_sql_query(query, connection)
df.head(10)

,pays,dispo_prot_tot
0,Libéria,13.74590
1,Guinée-Bissau,16.07825
2,Mozambique,16.67320
3,République centrafricaine,16.80460
4,Madagascar,17.04185
5,Haïti,17.41050
6,Zimbabwe,17.63680
7,Congo,18.76465
8,Ouganda,19.21360
9,Sao Tomé-et-Principe,19.38150


**La quantité totale (en kg) de produits perdus par pays en 2013.**

In [7]:
query = ''' SELECT pays, SUM(pertes*1000) AS pertes_totales FROM equilibre_prod
            GROUP BY pays
            ORDER BY pertes_totales DESC
            LIMIT 10 '''

df = pd.read_sql_query(query, connection)
df.head(10) 

,pays,pertes_totales
0,Chine,90358000.0
1,Brésil,75914000.0
2,Inde,55930000.0
3,Nigéria,19854000.0
4,Indonésie,13081000.0
5,Turquie,12036000.0
6,Mexique,8289000.0
7,Égypte,7608000.0
8,Ghana,7442000.0
9,États-Unis d'Amérique,7162000.0


**Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.**

In [12]:
query = ''' SELECT sous_nutrition.pays, ((nb_personnes / population)*100) AS pourcent_personnes 
            FROM sous_nutrition AS sous_nutrition
            JOIN population AS population
            ON sous_nutrition.pays = population.pays
            ORDER BY pourcent_personnes DESC
            LIMIT 10'''

df = pd.read_sql_query(query, connection)
df.head(10)

,pays,pourcent_personnes
0,Haïti,50.402249
1,Zambie,48.146365
2,Zimbabwe,46.643110
3,République centrafricaine,43.327556
4,République populaire démocratique de Corée,42.578831
5,Congo,40.467626
6,Tchad,38.206628
7,Angola,37.723547
8,Libéria,37.261295
9,Madagascar,35.768811


**Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.**

In [9]:
query = ''' SELECT produit, AVG(autres_utilisations/dispo_int) AS ratio
            FROM equilibre_prod
            WHERE dispo_int <> 0 AND autres_utilisations IS NOT NULL
            GROUP BY produit
            ORDER BY ratio DESC
            LIMIT 10 '''

df = pd.read_sql_query(query, connection)
df.head(10)

,produit,ratio
0,"Alcool, non Comestible",0.964602
1,Plantes Aquatiques,0.918852
2,Huile de Palme,0.650285
3,Huil Plantes Oleif Autr,0.549599
4,Huile de Palmistes,0.535026
5,Huile de Colza&Moutarde,0.458441
6,Huiles de Poissons,0.403712
7,Huile de Coco,0.368357
8,Graisses Animales Crue,0.305783
9,Manioc,0.230553


### Question 20 

**Pour quelques uns des produits identifiés dans cette dernière requête SQL, supposez quelles sont ces "autres utilisations" possibles (recherchez sur internet !)**

[Source autres utilisations Huile de Palme](https://fr.wikipedia.org/wiki/Huile_de_palme#Usages)

[Source autres utilisations Huile de Palme / Huile de Colza](https://www.capital.fr/auto/agrocarburants-le-match-huile-de-palme-colza-en-cinq-questions-1292378)

[Production Mondiale de biocarburant](https://www.planetoscope.com/biocarburants/217-production-mondiale-de-biocarburants.html)

[Souce Huile de Poissons](https://fr.wikipedia.org/wiki/Huile_de_poisson)

In [10]:
# À l'échelle mondiale, l’huile de palme est principalement utilisée dans l’industrie alimentaire (près de
# 70 %), dans l’oléochimie (savon, cosmétiques…) et en tant qu’agrocarburant (plus de 20 %). En Europe, la 
# moitié des importations d'huile de palme servent à fabriquer les biocarburants ; en France, cette part est
# même de 60 %. Avec plus de 50 millions de tonnes produites chaque année, c'est l'huile végétale la plus 
# consommée au monde (35 % de la consommation mondiale en 2017)

# L'huile de colza, est également utilisée dans la fabrication de biocarburant. 

# L'huile de Poissons peut être retrouvée dans l'industrie pharmaceutique. 